### https://www.tensorflow.org/text/tutorials/classify_text_with_bert

# 1. Sapm Detection

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import metrics
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
#  https://www.kaggle.com/uciml/sms-spam-collection-dataset
data = pd.read_csv(r'spam.csv', encoding='latin1')
print('sample number:',len(data))

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data

In [ ]:
data.drop_duplicates(subset=['v2'], inplace=True)

In [ ]:
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data = data[:200]
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

# numlist = list(range(len(data)))
# data = data.sample(400, random_state=100)
# numlist = list(range(len(data)))
# data = data.set_index(pd.Index(numlist))
# data

In [ ]:
data['v1'].value_counts()

In [ ]:
# data.drop(data[:380][data['v1'][:380] == 0].index, inplace=True)
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
X = data['v2']
y = data['v1']
y = y.astype(float)

In [ ]:
y.value_counts()

In [ ]:
print(len(X), X[0])

In [ ]:
# y = tf.convert_to_tensor(y, dtype=tf.float32, dtype_hint=None, name=None)

In [ ]:
print(len(y), y[0])

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
}

map_model_to_preprocess = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
#     net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)     # activation=None
    return tf.keras.Model(text_input, net)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 32
epochs = 10

from keras import optimizers

steps_per_epoch = int(len(X)*0.4)  # length of train data
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
X

In [ ]:
y_hard = pd.DataFrame(y)
y_hard

# 1-0. Generating Prob_labels

In [ ]:
X_gen = tf.convert_to_tensor(X, dtype=tf.string, dtype_hint=None, name=None)
gen_model = build_classifier_model()
opt = optimization.create_optimizer(init_lr=0.0001,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
gen_model.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
history = gen_model.fit(X_gen, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X_gen, verbose=0)
prob_label = list(result.reshape(len(X),))
prob_label

# 1-1. Focal(Hard) and SLS(Hard/diverse alphas)_option#1

In [ ]:
for t in range(10):    # 10 times repeat    
    res = pd.DataFrame({'Focal':[0, 0, 0, 0, 0]}, index = ['Acc','Pre','Rec','F1','R-AUC']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_acc = []
    list_pre = []
    list_rec = []
    list_f1 = []
    list_rauc = []   
    focal_model = build_classifier_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]
        X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)

        opt = optimization.create_optimizer(init_lr=0.0001,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = np.round(focal_model.predict(X_test, verbose=0))
        list_acc.append(metrics.accuracy_score(y_test, predicted))
        list_pre.append(metrics.precision_score(y_test, predicted))
        list_rec.append(metrics.recall_score(y_test, predicted))
        list_f1.append(metrics.f1_score(y_test, predicted))
        list_rauc.append(metrics.roc_auc_score(y_test, predicted))
    res['Focal'] = [np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)]
    print([np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)])
    
    B = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]  # SLS with LS
    c = 0.1  # criterion decides easy/hard
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_005 = []
        for i in range(len(y_hard)):
            if list(y_hard['v1'])[i] == 0:
                if prob_label[i] <= c:
                    y_005.append(b)  # easy sample
                else:
                    y_005.append(0) # (or 0-b) hard sample
            if list(y_hard['v1'])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_005.append(1-b)  # easy sample
                else:
                    y_005.append(1) # (or 1+b) hard sample
        y_005 = pd.DataFrame(y_005)     

        bce005_acc = []
        bce005_pre = []
        bce005_rec = []
        bce005_f1 = []
        bce005_rocauc = []
        model_005 = build_classifier_model()
#             early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        n_iter = 0
        for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
            n_iter += 1
            X_train = X[train_index]
            y_005_train= y_005.iloc[train_index]
            if n_iter == 1:
                print(y_005_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]  # test with real(actual) label y
            X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
            X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
            
            # MLP_BCE(y_005)
            opt = optimization.create_optimizer(init_lr=0.0001,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
            model_005.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
            history = model_005.fit(X_train, y_005_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            predicted = np.round(model_005.predict(X_test, verbose=0))
            bce005_acc.append(metrics.accuracy_score(y_test, predicted))
            bce005_pre.append(metrics.precision_score(y_test, predicted))
            bce005_rec.append(metrics.recall_score(y_test, predicted))
            bce005_f1.append(metrics.f1_score(y_test, predicted))
            bce005_rocauc.append(metrics.roc_auc_score(y_test, predicted))
        res['SLS({})'.format(b)] = [np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)]
        print([np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)])        
    res.to_csv("BERT_SPAM_5CV(SLS_opt#1_c0.1).csv", mode = 'a', float_format='%.4g')

In [ ]:
res = pd.read_csv("BERT_SPAM_5CV(SLS_opt#1_c0.1).csv")
res = res.dropna(axis=0)
res = res.rename(columns={'SLS(0.0)':'Hard'})
df_acc = res[res.iloc[:,0] == 'Acc']
df_acc = df_acc.reset_index(drop=True)
df_acc = df_acc.iloc[:,1:].astype(float)
col_name = df_acc.columns
ave = []
std = []
for i in range(12):
    ave.append(np.mean(list(df_acc.iloc[:,i])))
    std.append(np.std(list(df_acc.iloc[:,i])))
final = pd.DataFrame(ave, index=col_name, columns=["mean"])
final['std'] = std
final.sort_values("mean", ascending=False)

In [ ]:
len(df_acc)

# 2. Reuters News

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import metrics
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
data = pd.read_csv(r"ModHayes_train.csv")
data

In [ ]:
data = data[["text","topics"]]

In [ ]:
data['topics'].value_counts()[:10]

In [ ]:
isblank = data['topics'] == "[]"
isacq = data['topics'] == "['acq']"
data = data[isblank | isacq]
data = data.dropna()
data

In [ ]:
data['topics'] = data['topics'].replace(["[]","['acq']"],[0,1])
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data = data[:350]
data

In [ ]:
data['topics'].value_counts()

In [ ]:
data.drop(data[:295][data['topics'][:295] == 0].index, inplace=True)
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
data['topics'].value_counts()

In [ ]:
X = data['text']
y = data['topics']
y = y.astype(float)

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
}

map_model_to_preprocess = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
#     net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)     # activation=None
    return tf.keras.Model(text_input, net)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 64
epochs = 10

from keras import optimizers

steps_per_epoch = int(len(X)*0.4)  # length of train data
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
X

In [ ]:
y_hard = pd.DataFrame(y)
y_hard

# 2-0. Generating Prob_labels

In [ ]:
X_gen = tf.convert_to_tensor(X, dtype=tf.string, dtype_hint=None, name=None)
gen_model = build_classifier_model()
opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
gen_model.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
history = gen_model.fit(X_gen, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X_gen, verbose=0)
prob_label = list(result.reshape(len(X),))
prob_label

# 2-1. Focal(Hard) and SLS(Hard/diverse alphas)_option#1

In [ ]:
for t in range(10):    # 10 times repeat    
    res = pd.DataFrame({'Focal':[0, 0, 0, 0, 0]}, index = ['Acc','Pre','Rec','F1','R-AUC']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_acc = []
    list_pre = []
    list_rec = []
    list_f1 = []
    list_rauc = []   
    focal_model = build_classifier_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]
        X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)

        opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = np.round(focal_model.predict(X_test, verbose=0))
        list_acc.append(metrics.accuracy_score(y_test, predicted))
        list_pre.append(metrics.precision_score(y_test, predicted))
        list_rec.append(metrics.recall_score(y_test, predicted))
        list_f1.append(metrics.f1_score(y_test, predicted))
        list_rauc.append(metrics.roc_auc_score(y_test, predicted))
    res['Focal'] = [np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)]
    print([np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)])
    
    B = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]  # SLS with LS
    c = 0.1  # criterion decides easy/hard
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_005 = []
        for i in range(len(y_hard)):
            if list(y_hard['topics'])[i] == 0:
                if prob_label[i] <= c:
                    y_005.append(b)  # easy sample
                else:
                    y_005.append(0) # (or 0-b) hard sample
            if list(y_hard['topics'])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_005.append(1-b)  # easy sample
                else:
                    y_005.append(1) # (or 1+b) hard sample
        y_005 = pd.DataFrame(y_005)     

        bce005_acc = []
        bce005_pre = []
        bce005_rec = []
        bce005_f1 = []
        bce005_rocauc = []
        model_005 = build_classifier_model()
#             early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        n_iter = 0
        for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
            n_iter += 1
            X_train = X[train_index]
            y_005_train= y_005.iloc[train_index]
            if n_iter == 1:
                print(y_005_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]  # test with real(actual) label y
            X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
            X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
            
            # MLP_BCE(y_005)
            opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
            model_005.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
            history = model_005.fit(X_train, y_005_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            predicted = np.round(model_005.predict(X_test, verbose=0))
            bce005_acc.append(metrics.accuracy_score(y_test, predicted))
            bce005_pre.append(metrics.precision_score(y_test, predicted))
            bce005_rec.append(metrics.recall_score(y_test, predicted))
            bce005_f1.append(metrics.f1_score(y_test, predicted))
            bce005_rocauc.append(metrics.roc_auc_score(y_test, predicted))
        res['SLS({})'.format(b)] = [np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)]
        print([np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)])        
    res.to_csv("BERT_RNEWS_5CV(SLS_opt#1_c0.1).csv", mode = 'a', float_format='%.4g')

In [ ]:
res = pd.read_csv("BERT_RNEWS_5CV(SLS_opt#1_c0.1).csv")
res = res.dropna(axis=0)
res = res.rename(columns={'SLS(0.0)':'Hard'})
df_acc = res[res.iloc[:,0] == 'Acc']
df_acc = df_acc.reset_index(drop=True)
df_acc = df_acc.iloc[:,1:].astype(float)
col_name = df_acc.columns
ave = []
std = []
for i in range(12):
    ave.append(np.mean(list(df_acc.iloc[:,i])))
    std.append(np.std(list(df_acc.iloc[:,i])))
final = pd.DataFrame(ave, index=col_name, columns=["mean"])
final['std'] = std
final.sort_values("mean", ascending=False)

In [ ]:
len(df_acc)

# 3. IMDB Sentiment Analysis

In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tf.get_logger().setLevel('ERROR')

In [ ]:
data = pd.read_csv(r"IMDB Dataset.csv")
data

In [ ]:
data['sentiment'] = data['sentiment'].replace(['negative','positive'],[0,1])
data

In [ ]:
X_train = data['review'][:150]
y_train = data['sentiment'][:150]

In [ ]:
print(len(X_train), len(y_train))

In [ ]:
pd.Series(y_train).value_counts()

In [ ]:
# Making lists of index
idx_0 = []
idx_1 = []
for i in range(len(y_train)):
    if list(y_train)[i] == 0:
        idx_0.append(i)
    if list(y_train)[i] == 1:
        idx_1.append(i)
print(len(idx_0), len(idx_1))

In [ ]:
idx = idx_0[:50] + idx_1[:50]
idx.sort()
print(len(idx), idx)

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_train[i])
    y.append(y_train[i])
X = np.array(X)
y = np.array(y)
y = y.astype(float)
print(X.shape, y.shape)

In [ ]:
print(pd.Series(y).value_counts())

In [ ]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
}

map_model_to_preprocess = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

In [ ]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
#     net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)     # activation=None
    return tf.keras.Model(text_input, net)

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

batch = 32
epochs = 10

from keras import optimizers

steps_per_epoch = int(len(X)*0.4)  # length of train data
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
X = pd.Series(X)
X

In [ ]:
y_hard = pd.DataFrame(y)
y_hard

# 3-0. Generating Prob_labels

In [ ]:
X_gen = tf.convert_to_tensor(X, dtype=tf.string, dtype_hint=None, name=None)
gen_model = build_classifier_model()
opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
gen_model.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
history = gen_model.fit(X_gen, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X_gen, verbose=0)
prob_label = list(result.reshape(len(X),))
prob_label

# 3-1. Focal(Hard) and SLS(Hard/diverse alphas)_option#1

In [ ]:
for t in range(10):    # 10 times repeat    
    res = pd.DataFrame({'Focal':[0, 0, 0, 0, 0]}, index = ['Acc','Pre','Rec','F1','R-AUC']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_acc = []
    list_pre = []
    list_rec = []
    list_f1 = []
    list_rauc = []   
    focal_model = build_classifier_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]
        X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
        X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)

        opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=opt, metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = np.round(focal_model.predict(X_test, verbose=0))
        list_acc.append(metrics.accuracy_score(y_test, predicted))
        list_pre.append(metrics.precision_score(y_test, predicted))
        list_rec.append(metrics.recall_score(y_test, predicted))
        list_f1.append(metrics.f1_score(y_test, predicted))
        list_rauc.append(metrics.roc_auc_score(y_test, predicted))
    res['Focal'] = [np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)]
    print([np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)])
    
    B = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]  # SLS with LS
    c = 0.1  # criterion decides easy/hard
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_005 = []
        for i in range(len(y_hard)):
            if list(y_hard[0])[i] == 0:
                if prob_label[i] <= c:
                    y_005.append(b)  # easy sample
                else:
                    y_005.append(0) # (or 0-b) hard sample
            if list(y_hard[0])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_005.append(1-b)  # easy sample
                else:
                    y_005.append(1) # (or 1+b) hard sample
        y_005 = pd.DataFrame(y_005)     

        bce005_acc = []
        bce005_pre = []
        bce005_rec = []
        bce005_f1 = []
        bce005_rocauc = []
        model_005 = build_classifier_model()
#             early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        n_iter = 0
        for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
            n_iter += 1
            X_train = X[train_index]
            y_005_train= y_005.iloc[train_index]
            if n_iter == 1:
                print(y_005_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]  # test with real(actual) label y
            X_train = tf.convert_to_tensor(X_train, dtype=tf.string, dtype_hint=None, name=None)
            X_test = tf.convert_to_tensor(X_test, dtype=tf.string, dtype_hint=None, name=None)
            
            # MLP_BCE(y_005)
            opt = optimization.create_optimizer(init_lr=0.0002,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps,optimizer_type='adamw')
            model_005.compile(loss='BinaryCrossentropy', optimizer=opt, metrics=['accuracy'])
            history = model_005.fit(X_train, y_005_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            predicted = np.round(model_005.predict(X_test, verbose=0))
            bce005_acc.append(metrics.accuracy_score(y_test, predicted))
            bce005_pre.append(metrics.precision_score(y_test, predicted))
            bce005_rec.append(metrics.recall_score(y_test, predicted))
            bce005_f1.append(metrics.f1_score(y_test, predicted))
            bce005_rocauc.append(metrics.roc_auc_score(y_test, predicted))
        res['SLS({})'.format(b)] = [np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)]
        print([np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)])        
    res.to_csv("BERT_IMDB_5CV(SLS_opt#1_c0.1).csv", mode = 'a', float_format='%.4g')

In [ ]:
res = pd.read_csv("BERT_IMDB_5CV(SLS_opt#1_c0.1).csv")
res = res.dropna(axis=0)
res = res.rename(columns={'SLS(0.0)':'Hard'})
df_acc = res[res.iloc[:,0] == 'Acc']
df_acc = df_acc.reset_index(drop=True)
df_acc = df_acc.iloc[:,1:].astype(float)
col_name = df_acc.columns
ave = []
std = []
for i in range(12):
    ave.append(np.mean(list(df_acc.iloc[:,i])))
    std.append(np.std(list(df_acc.iloc[:,i])))
final = pd.DataFrame(ave, index=col_name, columns=["mean"])
final['std'] = std
final.sort_values("mean", ascending=False)

In [ ]:
len(df_acc)